In [1]:
import pandas as pd
from scapy.all import *
import plotly
import math

Шаг 1. Читаем полученный файл с перехваченными пакетами:

In [2]:
packets = rdpcap("traff.pcap")

Шаг 2. Ищем пакеты, в которых использовался UDP протокол:

In [3]:
pktBytes = []
pktTimes = []

for pkt in packets:
    if UDP in pkt:
        try:
            pktBytes.append(pkt[UDP].len)
            pktTime = datetime.fromtimestamp(pkt.time)
            pktTimes.append(pktTime.strftime("%H:%M:%S.%f"))
        except:
            pass

<ipython-input-3-095eb5127fb7>:8: DeprecationWarning: an integer is required (got type EDecimal).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  pktTime = datetime.fromtimestamp(pkt.time)


Шаг 3. Строим график зависимости количества отправленных байтов от времени:

In [4]:
import random
bytes = pd.Series(pktBytes).astype(int)
times = pd.to_datetime(pd.Series(pktTimes).astype(str),  errors = "coerce")
df  = pd.DataFrame({"Times":times, "Bytes": bytes})
df = df.set_index("Times")
df2 = df.resample("1S").sum()

plotly.offline.plot({
    "data":[plotly.graph_objs.Scatter(x = df2.index, y = df2["Bytes"])],
    "layout":plotly.graph_objs.Layout(title = "Распределение пакетов UDP ",
    xaxis = dict(title = "Время"),
    yaxis = dict(title = "Байты"))
})

'temp-plot.html'

Шаг 4. Проверяем справедливость гипотезы с помощью критерия Колмогорова-Смирнова:

In [5]:
byteCount = []
mean = 0

for i in range(len(df2)):
    byteCount.append(df2["Bytes"][i])
    mean += byteCount[i]
mean /= len(df2)
byteCountTheoria = []

for i in range(len(df2)):
    byteCountTheoria.append(mean)
byteCountNak = []

for i in range(len(df2)):
    if i == 0:
        byteCountNak.append(byteCount[i])
    else :
        byteCountNak.append(byteCount[i] + byteCountNak[i - 1])

byteCountTherNak = []
for i in range(len(df2)):
    if i == 0:
        byteCountTherNak.append(byteCountTheoria[i])
    else :
        byteCountTherNak.append(byteCountTheoria[i] + byteCountTherNak[i - 1])
        
byteCountPercent = []
for i in range(len(df2)):
    byteCountPercent.append(byteCountNak[i] / byteCountNak[len(df2) - 1])
    
byteCountTheorPercent = []
for i in range(len(df2)):
    byteCountTheorPercent.append(byteCountTherNak[i] / byteCountTherNak[len(df2) - 1])
    
byteCountPercentDiff = []
for i in range(len(df2)):
    byteCountPercentDiff.append(abs(byteCountPercent[i] - byteCountTheorPercent[i]))
    
maxDif = max(byteCountPercentDiff)
realTotal = byteCountNak[len(df2) - 1]
teorTotal = byteCountTherNak[len(df2) - 1]
criteriaManual = maxDif
pval = maxDif * (len(df2)) ** 0.5
print("Максимальная разница эмпирических функций распределения = ", criteriaManual)
print("Статистика критерия = ", pval)
alpha = 0.05
alphaCritical = 1.36
print("Уровень значимости равен = ", alpha)
print("Квантиль распределения Колмогорова для выборок больше 20 = ", alphaCritical)

if alphaCritical < pval :
    print("Статистика критерия превышает квантиль распределения Колмогорова")
    print("Гипотеза не справедлива")
else:
    print("Статистика критерия не превышает квантиль распределения Колмогорова")
    print("Гипотеза справедлива для данного уровня значимости")

Максимальная разница эмпирических функций распределения =  0.14889334802248677
Статистика критерия =  1.3482866108503488
Уровень значимости равен =  0.05
Квантиль распределения Колмогорова для выборок больше 20 =  1.36
Статистика критерия не превышает квантиль распределения Колмогорова
Гипотеза справедлива для данного уровня значимости
